In [25]:
import psycopg2
import pgvector
import pandas as pd
conn = psycopg2.connect(
    host="localhost",
    database="postgres",
    user="postgres",
    password="290128"
)

In [26]:
conn.autocommit=True
conn 
if(False):
    cur=conn.cursor()
    cur.execute("CREATE EXTENSİON vector;")

In [27]:
MODEL_NAME='all-MiniLM-L6-v2'
from sentence_transformers import SentenceTransformer
model=SentenceTransformer(MODEL_NAME)

In [39]:
docs= [ 
{"input":"We formed our partnership with OpenAI around a shared ambition to \ responsibly advance cutting-edge AI research and democratize AI as a new \ technology platform,” said Satya Nadella, Chairman and CEO, Microsoft. \ In this next phase of our partnership, developers and organizations across \ industries will have access to the best AI infrastructure, models, and toolchain \ with Azure to build and run their applications."}
]
print('len(docs)',len(docs))

len(docs) 1


In [29]:
documents=[]
ids=[]
for index,doc in enumerate(docs):
    doc_content=doc["input"]
    documents.append(doc_content)
    ids.append(str(index))
    embeddings=model.encode(documents)

In [30]:
embeddings[0].tolist()

[-0.026725545525550842,
 -0.054436445236206055,
 -0.047934334725141525,
 0.027443425729870796,
 0.09098034352064133,
 -0.013835313729941845,
 -0.005485240835696459,
 -0.04992423579096794,
 0.04061346501111984,
 0.009536135010421276,
 -0.061506278812885284,
 -0.0077025918290019035,
 0.032477278262376785,
 0.017658721655607224,
 0.0651795044541359,
 0.14944639801979065,
 0.008546381257474422,
 -0.029865946620702744,
 -0.09027165174484253,
 -0.010371614247560501,
 -0.037613723427057266,
 0.028216414153575897,
 -0.003459315747022629,
 -0.056050460785627365,
 0.03761869668960571,
 0.021069301292300224,
 0.011141099035739899,
 -0.06682039052248001,
 -0.0055345697328448296,
 0.0036667650565505028,
 -0.002282370114699006,
 -0.014770053327083588,
 0.05275994539260864,
 0.0360734686255455,
 0.030314330011606216,
 0.043572887778282166,
 -0.02920306846499443,
 -0.06029091775417328,
 -0.03818494454026222,
 -0.09580733627080917,
 0.059583377093076706,
 -0.025053158402442932,
 -0.02381063997745514,
 

In [31]:
DERIVED_EMB_SIZE=len(embeddings[0])
DERIVED_EMB_SIZE

384

In [32]:
if(False):
    cur=con.cursor()
    cur.execute("DROP TABLE IF EXISTS items2;")
    cur.close()

In [33]:
if(True):
    cur=conn.cursor()
    cur.execute("CREATE TABLE itemss (id bigserial PRIMARY KEY, text TEXT, embedding_vector BYTEA);")
    cur.close()


In [36]:
cur = conn.cursor()
for index, item in enumerate(documents):
    my_doc = {"id": index, "text": documents[index], "embedding": embeddings[index].tolist()}
    cur.execute("""INSERT INTO items3 (id, text, embedding_vector) VALUES (%(id)s, %(text)s, %(embedding)s)""", my_doc)

# Döngü tamamlandıktan sonra cursor nesnesini kapat
cur.close()

    

DatatypeMismatch: column "embedding_vector" is of type bytea but expression is of type numeric[]
LINE 1: ...with Azure to build and run their applications.', ARRAY[ -0....
                                                             ^
HINT:  You will need to rewrite or cast the expression.


In [ ]:
my_doc

{'id': 0,
 'text': 'We formed our partnership with OpenAI around a shared ambition to \\ responsibly advance cutting-edge AI research and democratize AI as a new \\ technology platform,” said Satya Nadella, Chairman and CEO, Microsoft. \\ In this next phase of our partnership, developers and organizations across\xa0\\ industries will have access to the best AI infrastructure, models, and toolchain \\ with Azure to build and run their applications.',
 'embedding': [-0.026725545525550842,
  -0.054436445236206055,
  -0.047934334725141525,
  0.027443425729870796,
  0.09098034352064133,
  -0.013835313729941845,
  -0.005485240835696459,
  -0.04992423579096794,
  0.04061346501111984,
  0.009536135010421276,
  -0.061506278812885284,
  -0.0077025918290019035,
  0.032477278262376785,
  0.017658721655607224,
  0.0651795044541359,
  0.14944639801979065,
  0.008546381257474422,
  -0.029865946620702744,
  -0.09027165174484253,
  -0.010371614247560501,
  -0.037613723427057266,
  0.028216414153575897,

In [37]:
cur=conn.cursor()
cur.execute("""SELECT count(*) as cnt FROM itemss LIMIT 10""")
for r in cur.fetchall():
    print(r)
cur.close()

(0,)


In [38]:
cur=conn.cursor()
cur.execute("""SELECT id,text,embedding FROM itemss LIMIT 10""")
for r in cur.fetchall():
    print(r[0])
cur.close()

UndefinedColumn: column "embedding" does not exist
LINE 1: SELECT id,text,embedding FROM itemss LIMIT 10
                       ^
